In [1]:
# https://k3no.medium.com/how-to-query-in-graphql-6ebb3f7085dc
# https://github.com/sushiswap/sushiswap-subgraph

In [1]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import dataframe_image as dfi
import os

fantom = 'https://api.thegraph.com/subgraphs/name/eerieeight/spooky-swap-exchange'
# https://thegraph.com/explorer/subgraph/eerieeight/spooky-swap-exchange

## Fantom

In [7]:
sample_transport=RequestsHTTPTransport(
    url=fantom,
    verify=True,
    retries=3,
)

client = Client(transport=sample_transport)

query = gql('''
query {
  uniswapFactories{
    pairCount
  }
}
''')

response = client.execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

394


In [15]:
token0list = []
token1list = []
id_list = []

for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0{symbol}
                      token1{symbol}
                      id
                  }
                }
                '''
        response = client.execute(gql(querystring))
        for row in response['pairs']:
            token0list.append(row['token0']['symbol'])
            token1list.append(row['token1']['symbol'])
            id_list.append(row['id'])    

df_spooky = pd.DataFrame.from_dict(list(zip(token0list,token1list,id_list)))
df_spooky.columns = ['Token 0', 'Token 1', 'id']
df_spooky['Pair'] = df_spooky['Token 0'] + '-' + df_spooky['Token 1']
df_spooky = df_spooky[['Pair', 'id', 'Token 0', 'Token 1']]
df_spooky

,Pair,id,Token 0,Token 1
0,WFTM-SFI,0x01424c64c4744769299019be64f3d82898ff28f3,WFTM,SFI
1,WFTM-FIVE,0x05286a1d7e82dc75a7485931394d328608075c9a,WFTM,FIVE
2,WFTM-SHADOW,0x05dee7aed28dd296742cfe4b5f0176b4619d8af8,WFTM,SHADOW
3,WFTM-SPIDER,0x064ec5dcac4be6b213e3434f1c9c7cd204b1cf02,WFTM,SPIDER
4,WFTM-SNX,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,WFTM,SNX
...,...,...,...,...
171,WFTM-eMax,0xf9d8d5210dc748514b856a0ef0a90089e811a96f,WFTM,eMax
172,WFTM-Ching,0xfb5bea4a59e8682a927d9ff1eababd9e2d1d37e7,WFTM,Ching
173,WFTM-WOOFY,0xfd0ab56b83130ce8f2b7a4f4d4532dee495c0794,WFTM,WOOFY
174,WFTM-BTC,0xfdb9ab8b9513ad9e419cf19530fee49d412c3ee3,WFTM,BTC


In [21]:
cream_borrowing_tokens = ['USDC', 'WFTM', 'DAI', 'ETH', 'BTC', 'LINK', 'SUSHI',\
'YFI',
'SNX',
'BAND',
'AAVE',
'COVER',
'CREAM',
'HEGIC']

interesting_tokens = ['WFTM', 'USDC', 'DAI', 'FUSD', 'ETH', 'fUSDT', 'FETH', 'WBTC']
combined_tokens = cream_borrowing_tokens + interesting_tokens
combined_tokens

['USDC',
 'WFTM',
 'DAI',
 'ETH',
 'BTC',
 'LINK',
 'SUSHI',
 'YFI',
 'SNX',
 'BAND',
 'AAVE',
 'COVER',
 'CREAM',
 'HEGIC',
 'WFTM',
 'USDC',
 'DAI',
 'FUSD',
 'ETH',
 'fUSDT',
 'FETH',
 'WBTC']

In [22]:
criteria = df_spooky['Token 0'].isin(combined_tokens) & \
           df_spooky['Token 1'].isin(combined_tokens)

df_spooky_filtered = df_spooky.loc[criteria,:]
df_spooky_filtered.columns = ['Pair', 'id', 'Token 0', 'Token 1']
df_spooky_filtered

,Pair,id,Token 0,Token 1
4,WFTM-SNX,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,WFTM,SNX
32,USDC-WFTM,0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c,USDC,WFTM
33,DAI-FUSD,0x2c5515487eb4a8573af5432606e74fb70b45cac1,DAI,FUSD
50,USDC-DAI,0x484237bc35ca671302d19694c66d617142fbc235,USDC,DAI
51,fUSDT-DAI,0x4a217fbf837c6115b183eaac0245d6a72e8a98e4,fUSDT,DAI
60,fUSDT-WFTM,0x5965e53aa80a0bcf1cd6dbdd72e6a9b2aa047410,fUSDT,WFTM
64,WFTM-COVER,0x5dc7848bf215f1d99f2af3d2bf78fcdf238ee34b,WFTM,COVER
90,WFTM-LINK,0x89d9bc2f2d091cfbfc31e333d6dc555ddbc2fd29,WFTM,LINK
93,WFTM-FUSD,0x8ec0417a4c795074f076519afd4410484c9830ef,WFTM,FUSD
97,WFTM-BAND,0x91b39d5584e2a7dc829f696235742cc293f2e8cf,WFTM,BAND


In [26]:
df4 = pd.DataFrame([])
lenid = len(df_spooky_filtered['id'])

client = Client(transport=sample_transport)

for count, id in enumerate(df_spooky_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 7) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client.execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)
    
df4

 1 / 22 pair: 0x06d173628be105fe81f1c82c9979ba79ebcafcb7
 2 / 22 pair: 0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c
 3 / 22 pair: 0x2c5515487eb4a8573af5432606e74fb70b45cac1
 4 / 22 pair: 0x484237bc35ca671302d19694c66d617142fbc235
 5 / 22 pair: 0x4a217fbf837c6115b183eaac0245d6a72e8a98e4
 6 / 22 pair: 0x5965e53aa80a0bcf1cd6dbdd72e6a9b2aa047410
 7 / 22 pair: 0x5dc7848bf215f1d99f2af3d2bf78fcdf238ee34b
 8 / 22 pair: 0x89d9bc2f2d091cfbfc31e333d6dc555ddbc2fd29
 9 / 22 pair: 0x8ec0417a4c795074f076519afd4410484c9830ef
10 / 22 pair: 0x91b39d5584e2a7dc829f696235742cc293f2e8cf
11 / 22 pair: 0xa572bdf049382f1f98f9a430788dadd51a303969
12 / 22 pair: 0xb77b223490e1f5951ec79a8d09db9eab2adcb934
13 / 22 pair: 0xbf4d61d4cec3a9dff7452d8987e1cc2943e2eb4c
14 / 22 pair: 0xd92206379bd8203ac38225af006bb96bf1f12412
15 / 22 pair: 0xde1a99d2471470c33d9130c5dd485b0c99f3d0df
16 / 22 pair: 0xe120ffbda0d14f3bb6d6053e90e63c572a66a428
17 / 22 pair: 0xebf374bb21d83cf010cc7363918776adf6ff2bf6
18 / 22 pair: 0xec454eda10accdd

,dailyVolumeUSD,date,reserveUSD,id
0,71086.88927532650060962491797523849,1624060800,1173018.092097817841307525804403724,0x06d173628be105fe81f1c82c9979ba79ebcafcb7
1,104812.3101960717037297562195055202,1623974400,946995.1581472733097561422416498988,0x06d173628be105fe81f1c82c9979ba79ebcafcb7
2,96711.72711988294161165026281033622,1623888000,1001152.897112580438207825314934092,0x06d173628be105fe81f1c82c9979ba79ebcafcb7
3,48770.54707306825803765918857825352,1623801600,1084615.100288703268567497755322196,0x06d173628be105fe81f1c82c9979ba79ebcafcb7
4,292155.0266507534116257326616643006,1623715200,883710.6422447446215739402741747766,0x06d173628be105fe81f1c82c9979ba79ebcafcb7
...,...,...,...,...
4,367673.1756628642462489483822801758,1623715200,3002069.572735500983707062261878047,0xfdb9ab8b9513ad9e419cf19530fee49d412c3ee3
5,516633.8813533719272892427239425953,1623628800,3281509.04882307695773938269747905,0xfdb9ab8b9513ad9e419cf19530fee49d412c3ee3
6,368988.4400676916308391454959946907,1623542400,3125943.961857479565596368747207757,0xfdb9ab8b9513ad9e419cf19530fee49d412c3ee3
0,4.084971876074541986918876785826216,1622937600,0.002007135817760529857146729126256486,0xfdef392adc84607135c24ca45de5452d77aa10de


In [30]:
df5 = df4.merge(df_spooky_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['dailyVolumeUSD','reserveUSD']] = df5[['dailyVolumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s')
df5['fee'] = round(df5['dailyVolumeUSD'] * 0.003,6)
df5['1y APR for 100 invested'] = round((100/df5['reserveUSD']) * df5['fee'] * 365,3)
df5['Pair'] = '[Spooky Fantom] ' + df5['Pair']
df5.style.format({'reserveUSD': "{:0<4,.10f}"})

,dailyVolumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,1y APR for 100 invested
0,71086.889275,2021-06-19 00:00:00,"1,173,018.0920978179",0x06d173628be105fe81f1c82c9979ba79ebcafcb7,[Spooky Polygon] WFTM-SNX,WFTM,SNX,213.260668,6.636000
1,104812.310196,2021-06-18 00:00:00,"946,995.1581472734",0x06d173628be105fe81f1c82c9979ba79ebcafcb7,[Spooky Polygon] WFTM-SNX,WFTM,SNX,314.436931,12.119000
2,96711.727120,2021-06-17 00:00:00,"1,001,152.8971125805",0x06d173628be105fe81f1c82c9979ba79ebcafcb7,[Spooky Polygon] WFTM-SNX,WFTM,SNX,290.135181,10.578000
3,48770.547073,2021-06-16 00:00:00,"1,084,615.1002887033",0x06d173628be105fe81f1c82c9979ba79ebcafcb7,[Spooky Polygon] WFTM-SNX,WFTM,SNX,146.311641,4.924000
4,292155.026651,2021-06-15 00:00:00,"883,710.6422447446",0x06d173628be105fe81f1c82c9979ba79ebcafcb7,[Spooky Polygon] WFTM-SNX,WFTM,SNX,876.465080,36.201000
5,316680.612009,2021-06-14 00:00:00,"1,053,289.3991385687",0x06d173628be105fe81f1c82c9979ba79ebcafcb7,[Spooky Polygon] WFTM-SNX,WFTM,SNX,950.041836,32.922000
6,95828.846940,2021-06-13 00:00:00,"1,043,539.3539390548",0x06d173628be105fe81f1c82c9979ba79ebcafcb7,[Spooky Polygon] WFTM-SNX,WFTM,SNX,287.486541,10.055000
7,2277863.323855,2021-06-19 00:00:00,"4,766,400.2072892943",0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c,[Spooky Polygon] USDC-WFTM,USDC,WFTM,6833.589972,52.330000
8,3879001.994767,2021-06-18 00:00:00,"4,298,165.0888999999",0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c,[Spooky Polygon] USDC-WFTM,USDC,WFTM,11637.005984,98.821000
9,3425903.891773,2021-06-17 00:00:00,"4,453,819.3066776888",0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c,[Spooky Polygon] USDC-WFTM,USDC,WFTM,10277.711675,84.228000


In [32]:
criteria1 = df5['reserveUSD'] > 1
criteria2 = df5['date'] == df5['date'].max()

latest_results = df5.loc[(criteria1 & criteria2), :].sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'])\
                    .sort_values(by = '1y APR for 100 invested',
                                 ascending = False)\
                    .reset_index(drop = True)\
                    .drop(columns = ['id', 'Token 0', 'Token 1'])\
                    .reset_index(drop = True)[['date', 'Pair', '1y APR for 100 invested',  'reserveUSD', 'dailyVolumeUSD', 'fee']]

df_styled  = latest_results.style.format({'reserveUSD': "{:0<4,.2f}",
                             'dailyVolumeUSD': "{:0<4,.2f}",
                             'fee': "{:0<4,.2f}",
                             '1y APR for 100 invested': "{:0<4,.2f}"})\
              .set_properties(subset=["Pair"], **{'text-align': 'left'})

df_styled

,date,Pair,1y APR for 100 invested,reserveUSD,dailyVolumeUSD,fee
0,2021-06-19 00:00:00,[Spooky Polygon] USDC-WFTM,52.33,"4,766,400.21","2,277,863.32","6,833.59"
1,2021-06-19 00:00:00,[Spooky Polygon] fUSDT-WFTM,28.59,"3,353,857.42","875,734.07","2,627.20"
2,2021-06-19 00:00:00,[Spooky Polygon] WFTM-DAI,19.04,"3,990,883.98","693,992.23","2,081.98"
3,2021-06-19 00:00:00,[Spooky Polygon] WFTM-ETH,15.11,"3,410,404.18","470,730.48","1,412.19"
4,2021-06-19 00:00:00,[Spooky Polygon] USDC-ETH,12.96,"140,177.42","16,587.53",49.76
5,2021-06-19 00:00:00,[Spooky Polygon] WFTM-BTC,7.67,"3,830,104.14","268,099.75",804.30
6,2021-06-19 00:00:00,[Spooky Polygon] WFTM-SNX,6.64,"1,173,018.09","71,086.89",213.26
7,2021-06-19 00:00:00,[Spooky Polygon] WFTM-CREAM,5.74,"724,079.42","37,952.16",113.86
8,2021-06-19 00:00:00,[Spooky Polygon] WFTM-SUSHI,5.28,"1,586,824.70","76,509.73",229.53
9,2021-06-19 00:00:00,[Spooky Polygon] WFTM-LINK,4.95,"1,446,546.60","65,372.11",196.12


In [33]:
dfi.export(df_styled, 'df_styled_spooky_fantom.png')
os.startfile('df_styled_spooky_fantom.png')